In [1]:
%cd /home/q123/Desktop/explo

import torch 
import gpytorch 
import logging
import logging.config

import yaml


from src.config import insertion_config,insert
from src.helpers import setup_experiment
from src.trainer import Trainer
from src.gibo.optim import GIBOptimizer
from src.optim import BOptimizer

logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Cartpole

# env_config = {
#         "reward_scale":500,
#         "reward_shift":0,
#         "env_name":"CartPole-v1",
# }

# Swimmer
env_config = {
        "reward_scale":350,
        "reward_shift":0,
        "env_name":"Swimmer-v2",
}

# ## hopper
# env_config = {
#         "reward_scale":1000,
#         "reward_shift":1,
#         "env_name":"Hopper-v2",
# }


In [6]:

n_init = 1

likelihood_config = {
        "noise_hyperprior":gpytorch.priors.torch_priors.UniformPrior(a=0.01,b=0.3),
        "noise_constraint":gpytorch.constraints.constraints.Interval(1e-2,0.3)
        #"noise_constraint":gpytorch.constraints.GreaterThan(0.1)
        }

kernel_config = {

        "kernel_name":"rbf",
        
        ### default values 
        "N_max":10e6,
        "ard_num_dims":1000,
        "use_ard":True,
        
        ###### Cartpole
        # "ard_num_dims":5,
        # "N_max":20,

        ###### Hopper
        # "ard_num_dims":36,
        # "N_max":48,
        
        ##### Swimmer
        "ard_num_dims":18,
        "N_max":32,
        #########
        
        "lengthscale_hyperprior":gpytorch.priors.torch_priors.GammaPrior(3.0,6.0),
        "lengthscale_constraint":gpytorch.constraints.constraints.GreaterThan(0.001),
        "outputscale_constraint":gpytorch.constraints.constraints.GreaterThan(0.1),
        "outputscale_hyperprior":gpytorch.priors.torch_priors.NormalPrior(loc=2.0,scale=1.0),
        }

trainer_config = {
        "n_steps" :50 ,
        "report_freq":10,
        "save_best":True,
}

optimizer_config = {
        "n_eval":2,
        ### for GIBO
        "n_info_samples":8,
        "delta":0.1,
        "normalize_gradient":True,
        "standard_deviation_scaling":False,
}

In [7]:


model,objective_env = setup_experiment(env_config,kernel_config,likelihood_config,n_init)
### Chose optimizer 
#optimizer = BOptimizer(**optimizer_config)
optimizer = GIBOptimizer(model,**optimizer_config)
trainer = Trainer(model,objective_env,optimizer,**trainer_config)
rslt= trainer.run()

MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
 Using 32 last points to compute GP
current 0.1745128333568573 / max 1.009308934211731 /batch_mean 0.8068991899490356 /batch_max 0.9809073805809021 
##############################
covar_lengthscale max 0.4838898479938507 / min 0.2464846968650818                  covar_outputscale 0.10000003129243851                 noise 0.010000672191381454
##############################
current 1.0091722011566162 / max 1.0278239250183105 /batch_mean 0.9725052714347839 /batch_max 1.0174894332885742 
##############################
covar_lengthscale max 0.5098865032196045 / min 0.21342861652374268                  covar_outputscale 0.10000012814998627                 noise 0.010000655427575111
##############################
current 0.9150869846343994 / max 1.0361478328704834 /batch_mean 0.8837372064590454 /batch_max 0.9979345202445984 
##############################
covar_lengthscale max 0.5513732433319092 / min 0.4921071231365204                  

In [5]:
from src.tester import Tester 

model,objective_env = setup_experiment(env_config,kernel_config,likelihood_config,n_init)
tester = Tester(model,objective_env,
                 "/home/q123/Desktop/explo/local_optima/CartPole-v1",1e-1,
                 n_train=100,n_test=100)

train_data,train_pred,test_data,test_pred,tmp = tester.run()

MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]


/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment Swimmer-v2 is out of date. You should consider upgrading to version `v3` with the environment ID `Swimmer-v3`.
  logger.warn(


KeyError: 'N_max'